<a href="https://colab.research.google.com/github/hanbun-pom/siosio/blob/master/Yukapedia_Git%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GitHub参照版

不特定多数への公開は今の所ご遠慮下さい  
当ページ外で行われた操作についての責任は負いかねます

>～使い方～  
>①左上　PlayBackgroundを選択  
>②左端の▶を押して実行

2020/03/19 追加　完全一致検索  正規表現検索　動画URL生成  
2020/04/06 試作　GitHub版  

In [185]:
import pandas as pd
import datetime

# 配信一覧　取得
master = 'https://raw.githubusercontent.com/hanbun-pom/siosio/master/'
youtube = 'https://youtu.be/'
videos = pd.read_csv(master + 'videos.csv')

# コメント一覧　作成
df_list=[]
for Id in videos['videoId']:
    tmp_df = pd.read_table(master + Id + '.tsv', header=None, names=['time','name','comment','videoId'],quotechar='"', quoting=3)
    tmp_df['videoId']= Id
    df_list.append(tmp_df)
df = pd.concat(df_list, ignore_index=True)

# 配信タイトル、公開時間
df = df.merge(videos)
df['DateTime'] = pd.to_datetime(df['DateTime']).dt.date

# 再生時間指定URL
trans = str.maketrans({':': '',',': ''})
s = df['time'].str.translate(trans).astype(int)
s = s-40*(abs(s)//100)-2400*(abs(s)//10000)+再生開始時間
s = ((abs(s)+s)//2).astype(str)

df['videoId'] = youtube + df['videoId'] + '?t=' + s + 's'

print('コメント取得終了')
df

コメント取得終了


,time,name,comment,videoId,title,DateTime
0,-1:41,如水,待機,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
1,-1:37,紫ユウキ,しおしおしてきた！,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
2,-1:37,暇つぶし,待機,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
3,-1:34,si n,しおしおっ,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
4,-1:34,pv order,待機🌸,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06
...,...,...,...,...,...,...
796060,2:05:39,はやち,ありがとうーー,https://youtu.be/LZVdK5kgSOk?t=7509s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24
796061,2:05:39,チヒロ氏,お疲れ様でした！,https://youtu.be/LZVdK5kgSOk?t=7509s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24
796062,2:05:40,ふむふむ,あかん涙出てきた,https://youtu.be/LZVdK5kgSOk?t=7510s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24
796063,2:05:40,法条裕一,お疲れさま～,https://youtu.be/LZVdK5kgSOk?t=7510s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24


Timestamp('2020-04-06 00:00:00')

In [0]:
#「名前」「コメント」に特定のキーワードを含むコメントを抽出

名前 = ""  #@param {type:"string"}
名前完全一致 = False  #@param {type:"boolean"} 
名前正規表現 = False  #@param {type:"boolean"} 
コメント = ""  #@param {type:"string"}
コメント完全一致 = False  #@param {type:"boolean"}
コメント正規表現 = False  #@param {type:"boolean"} 

再生開始時間 =  -30#@param {type:"integer"}
日付範囲

def query_condition(data, col, query, is_perfect, is_regex):
    if is_perfect:
        return data[col].str.match(query) if is_regex else data[col] == query
    else:
        return data[col].str.contains(query, na=False, regex=is_regex)

name_condition = query_condition(df, "name", 名前, 名前完全一致, 名前正規表現)
comment_condition = query_condition(df, "comment", コメント, コメント完全一致, コメント正規表現)

df2 = df[name_condition & comment_condition].copy()

pd.set_option('display.max_rows', 10000)
df2

,time,name,comment,videoId,title,publishedAt
0,-1:41,如水,待機,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06T15:14:52.000Z
1,-1:37,紫ユウキ,しおしおしてきた！,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06T15:14:52.000Z
2,-1:37,暇つぶし,待機,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06T15:14:52.000Z
3,-1:34,si n,しおしおっ,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06T15:14:52.000Z
4,-1:34,pv order,待機🌸,https://youtu.be/gY1g0LbsMQw?t=0s,【テトリス99】4列RENとやらを練習してみたい【視聴者参加型】,2020-04-06T15:14:52.000Z
...,...,...,...,...,...,...
796060,2:05:39,はやち,ありがとうーー,https://youtu.be/LZVdK5kgSOk?t=7509s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24T12:14:30.000Z
796061,2:05:39,チヒロ氏,お疲れ様でした！,https://youtu.be/LZVdK5kgSOk?t=7509s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24T12:14:30.000Z
796062,2:05:40,ふむふむ,あかん涙出てきた,https://youtu.be/LZVdK5kgSOk?t=7510s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24T12:14:30.000Z
796063,2:05:40,法条裕一,お疲れさま～,https://youtu.be/LZVdK5kgSOk?t=7510s,【麻雀】#天鳳クリスマス杯【🎄🎅🎁】,2019-12-24T12:14:30.000Z


In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os

# %%timeit
# GoogleDriveのファイルを参照する
from google.colab import drive 
drive.mount('/content/drive')

# 全コメントデータを読み込む　（少し時間がかかります)
# 特定の文字が入っているコメントはバグるのでquotecharで指定して無視
DATA_DIR='/content/drive/My Drive/Colab Notebooks/data/'

# DATA_DIR='data'

files = glob.glob(os.path.join(DATA_DIR,'*.tsv'))
df_list=[]
for file in files:
    tmp_df = pd.read_table(file, header=None, names=['time','name','comment','spchat'],quotechar='"', quoting=3)
    tmp_df['URL'] = os.path.basename(file)
    df_list.append(tmp_df)
df = pd.concat(df_list, ignore_index=True)

#「名前」「コメント」に特定のキーワードを含むコメントを抽出

名前 = " "  #@param {type:"string"}
名前完全一致 = False  #@param {type:"boolean"} 
名前正規表現 = False  #@param {type:"boolean"} 
コメント = ""  #@param {type:"string"}
コメント完全一致 = False  #@param {type:"boolean"}
コメント正規表現 = False  #@param {type:"boolean"} 

再生開始時間 =  -30#@param {type:"integer"}

def query_condition(data, col, query, is_perfect, is_regex):
    if is_perfect:
        return data[col].str.match(query) if is_regex else data[col] == query
    else:
        return data[col].str.contains(query, na=False, regex=is_regex)

name_condition = query_condition(df, "name", 名前, 名前完全一致, 名前正規表現)
comment_condition = query_condition(df, "comment", コメント, コメント完全一致, コメント正規表現)

    
df2 = df[name_condition & comment_condition].copy()

# time, file(videoID) → 再生時間指定URL
# start, end指定URLで、動画の一部だけ切り抜くのもいいかも
s = df2['time'].str.replace(':', '').astype(int)
s = s-40*(abs(s)//100)-2400*(abs(s)//10000)+再生開始時間
s = ((abs(s)+s)//2).astype(str)

df2['URL']='https://youtu.be/'+df2['URL'].str.replace('.tsv','?t=')+s+'s'
pd.set_option('display.max_rows', 10000)

df2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,time,name,comment,spchat,URL
1,-0:09,kyo sirogumi,ふるえてきた,NaN,https://youtu.be/LZVdK5kgSOk?t=0s
5,-0:08,me ro,来るぞ！,NaN,https://youtu.be/LZVdK5kgSOk?t=0s
8,-0:07,maphy 1125,待機,NaN,https://youtu.be/LZVdK5kgSOk?t=0s
9,-0:07,moti ancoro,しおちゃんをケーキのサンタさんと入れ替えていいですか？いいよ。ありがと,NaN,https://youtu.be/LZVdK5kgSOk?t=0s
19,-0:05,nigo nigo,初配信だから緊張してるだろうな～,NaN,https://youtu.be/LZVdK5kgSOk?t=0s
...,...,...,...,...,...
901918,57:51,O T,あきらめよう！,NaN,https://youtu.be/alQe0l2uEqQ?t=3441s
901925,57:53,F 7,開けないと帰れないよ,NaN,https://youtu.be/alQe0l2uEqQ?t=3443s
901926,57:53,ちゃたろう ねこにゃ,いやいやいやいや,NaN,https://youtu.be/alQe0l2uEqQ?t=3443s
901927,57:53,bon bon,かわいい,NaN,https://youtu.be/alQe0l2uEqQ?t=3443s
